In [1]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import *
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

2023-04-17 14:44:20.081992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 14:44:20.177097: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-17 14:44:20.566211: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2023-04-17 14:44:20.566261: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_p

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-17 14:44:21.138337: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 14:44:21.138530: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 14:44:21.138654: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 14:44:21.510463: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 14:44:21.510634: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


# Goal: Estimate acceleration profile $\hat{m}$ Using Newton-Raphson to compensate for distortion between similar point cloud

Through the first order Taylor approximation we define: 

$\mathbf{h}(\hat{m})$ as the means of voxel centers in cloud $j$ passed through the distortion operator $\mathbb{C}$ according to estimated motion profile $\hat{m}$. 

$\mathbf{H}$ is the Jacobian of $\mathbf{h}$, which is multiplied by a linear correction factor $\delta m$.

$    y_i = \mathbf{h}(\hat{m}) + \mathbf{H} \delta m $


In [63]:
#load point cloud
# no distortion
# old_cloud =  np.load("sample_data/paper_figures/case1/raw_frame_0.npy")
# m_hat = np.array([-3., -2., 0., 0., 0., 0.0])

# # movement in x
# old_cloud =  np.load("sample_data/paper_figures/case2/raw_frame_3.npy") 
# m_hat = np.array([3, 0, 0., 0., 0., 0])
# # m_hat = np.array([3, 0, 0., 0., 0., -0.2]) #FOR DEBUG-- deform just a little
# gt =  np.load("sample_data/paper_figures/case2/base_vel_2.npy")

# movement in x, y, & yaw
old_cloud =  np.load("sample_data/paper_figures/case3/raw_frame_1.npy") 
m_hat = np.array([3, -1, 0., 0., 0., -1])
gt =  np.load("sample_data/paper_figures/case3/base_vel_2.npy")
# print(gt) 

# period_lidar = 1
# t_scale = (2*np.pi)/(-m_hat[-1] + (2*np.pi/period_lidar))
# print(t_scale)
# m_hat = m_hat*t_scale
# # m_hat[-1] = m_hat[-1]*t_scaled
# print(m_hat)


#downsample
old_cloud = old_cloud[::5,:]

In [64]:
from remove_motion_basic import linear_correction_old as lc 

new_cloud = lc(old_cloud, m_hat)
# undistorted = lc(new_cloud, -m_hat)

In [65]:
plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
# disp.append(Points(undistorted, c = "#CB2314"))
# disp.append(Points(old_cloud, c = "#CB2314"))
disp.append(Points(new_cloud, c = "#3F5151"))
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

# Analytically calculating derivatives with SymPy

In [5]:
from sympy import *

x, y, z = symbols('x y z')

init_printing(use_unicode=True)

In [6]:
#sovles for the derivative
ans1 = diff(cos(x), x)
print(ans1)

#creates an unevaluated derivative
ans2 = Derivative(cos(x), x)
print(ans2)

-sin(x)
Derivative(cos(x), x)


In [7]:
ans3 = integrate(sin((x+1)/2)*(cos(x)-sin(x))**2, x)
# print(latex(ans3)) #-> converts to LaTeX so I don't have to re-copy everything 

integrate(sin((x+1)/2)*(cos(x)-sin(x))**2, x)


In [8]:
a = Matrix([[10*x, 2], 
            [3, 4*x]])
# A = integrate(a**2)
# print(latex(A))
integrate(a**2)

⎡     3                   ⎤
⎢100⋅x                2   ⎥
⎢────── + 6⋅x     14⋅x    ⎥
⎢  3                      ⎥
⎢                         ⎥
⎢                  3      ⎥
⎢       2      16⋅x       ⎥
⎢   21⋅x       ───── + 6⋅x⎥
⎣                3        ⎦

In [9]:
b = Matrix([[x + 1, 2*y],
            [3*x**2*y, 3 + x + y**2]])
diff(b, y, x)

⎡ 0   0⎤
⎢      ⎥
⎣6⋅x  0⎦

# Derive Jacobain H

### $\mathbf{H} \in \mathbb{R}^{4N \times 6} $

### $\mathbf{H} = 
\begin{bmatrix}
^{(n)}H_{x} ~ ^{(n)}H_{y} ~ ^{(n)}H_{z} ~ ^{(n)}H_{\phi} ~ ^{(n)}H_{\theta} ~ ^{(n)}H_{\psi}
\end{bmatrix}$

### $ ^{(n)}\mathbf{H}_x =  \frac{\delta{\mathbb{C}}}{\delta x} ~ ^{(n)}y_j \in \mathbb{R}^{4 \times 1}$

### Here $^{(n)}y_j$ refers to distribution mean from scan j inside voxel n, denoted in homogenous coordinates

$[x, y, z] \rightarrow [x, y, z, 1] $ 

<!-- <span style="color:red"> TODO: figure out if I need to use homogeneous coordinates for $y_j$ </span> -->

# Calculate correction  function $\mathbb{C}$ 

## $\mathbb{C}(\hat{m}) = T_{\text{rect}}(M(\hat{m})) \in \mathbb{R}^{4 \times 4} $

Here, the rectifying transformation $T_{\text{rect}}$ for each point in $y_j$ depends on scaled values of $\hat{m}$ which are concatenated to form matrix $M$. Since points are stored in the order they are recieved and are somewhat even spread throughout the scene, we can approximate $M$ through a simple linspace matrix without needing to transform to a spherical representation. $t_{\text{scale}}$ is a scaling parameter to normalize the turn rate of the sensor in the presence of composite yaw rotation.

## $
M = \bigg{(} \{ r: r = t_{\text{scale}} \frac{p}{P}, p \in \{ 0, 1, ..., P \} \} \bigg{)} \hat{m}
\in \mathbb{R}^{P \times 6}
% \begin{bmatrix}
% 1 & 1 & 1 & 1 
% \end{bmatrix}
$

## $t_{\text{scale}} = \frac{2\pi}{-\psi + \omega_{\text{LIDAR}} } $ 


My old way of "unwinding" was acomplished by converting the point cloud to spherical coordinates, rescaling the yaw angle, and then converting back to Cartesian 

<!-- ## $y_{j, \text{unwound}} =
\text{s2c} \bigg{(} \text{c2s}(y_{j})
\begin{bmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & \psi_{\text{rect}}
\end{bmatrix} \bigg{)}$
 -->
Unfortunately, c2s() is a nonlinear operation so it makes things a little tricky when we need to do everything via linear algebra

<!-- ## $ \text{c2s}(y_j) = 
\begin{bmatrix}
x_1 & y_1 & z_1 \\
x_2 & y_2 & z_2 \\
 & \vdots &
\end{bmatrix}
\rightarrow
\begin{bmatrix}
r_1 & \phi_1 & \theta_1 \\
r_2 & \phi_2 & \theta_2 \\
r_3 & \phi_3 & \theta_3
\end{bmatrix}
$
 -->
<!-- ### GPT:
lol it really got this wro

$ ^{(n)}y_j = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}
$

$ B = \begin{bmatrix}
\text{sin}(\theta) ~ \text{cos}(\phi) & \text{cos}(\theta) ~ \text{cos}(\phi) & -\text{sin}(\phi) \\
\text{sin}(\theta)\text{sin}(\phi) & \text{cos}(\theta)\text{sin}(\phi) & \text{cos}(\phi) \\
\text{cos}(\theta) & -\text{sin}(\theta) & 0
\end{bmatrix}
$

$B ~ ^{(n)}y_j = ^{(n)}y_{j, \text{spherical}} $ -->


<!-- Putting everything together:

 $
\mathbb{C}(y_j, \hat{m}) =
M ~ T_{\text{rect}} \bigg{[} \text{s2c} \bigg{(} \text{c2s}(y_{j})
\begin{bmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & \psi_{\text{rect}}
\end{bmatrix} \bigg{)},
\begin{bmatrix}
    1 \\
    \vdots 
\end{bmatrix}
\bigg{]}^T$
 -->

In [10]:
period_lidar = 1
m_hat = np.array([3, 0, 0., 0., 0., -0.5])
t_scale = (2*np.pi)/(-m_hat[-1] + (2*np.pi/period_lidar))
# print(t_scale)
lsvec = np.linspace(0,t_scale, 100)

print(np.shape(m_hat))
M = m_hat * np.array([lsvec, lsvec, lsvec, lsvec, lsvec, lsvec]).T
# print(np.shape(M))
# print(M)

(6,)


## Get transform $T_{\text{rect}}$

In [11]:
x, y, z, phi, theta, psi = symbols('x y z phi theta psi')
# m_hat = Matrix([x, y, z, phi, theta, phi]).T
m_hat = np.array([1,2,3,0.4,0.5,0.6])
# print( "m_hat = ", pretty(m_hat))

T_roll = Matrix([[1, 0, 0, 0],
                 [0, cos(phi), sin(phi), 0],
                 [0, -sin(phi), cos(phi), 0],
                 [0, 0, 0, 1]])
# print("\n T_roll =") 
# print(pretty(T_roll))

T_pitch = Matrix([[cos(theta), 0, -sin(theta), 0],
                 [0, 1, 0, 0],
                 [sin(theta), 0, cos(theta), 0],
                 [0, 0, 0, 1]])
# print("\n T_pitch =") 
# print(pretty(T_pitch))

#Recall we are setting T_yaw to identity here since scan axis-aligned motion is already compensated for
T_yaw = Matrix([[cos(psi), sin(psi), 0, 0],
                 [-sin(psi), cos(psi), 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
# print("\n T_yaw =") 
# print(pretty(T_yaw))

T_trans = Matrix([[1, 0, 0, x],
                 [0, 1, 0, y],
                 [0, 0, 1, z],
                 [0, 0, 0, 1]])
# print("\n T_trans =") 
# print(pretty(T_trans))

# T_rect = (T_roll * T_pitch * T_yaw * T_trans)
T_rect = (T_roll * T_pitch * T_yaw * T_trans).inv()
print("T_rect =")
T_rect


T_rect =


⎡                                                                             
⎢                            cos(ψ)⋅cos(θ)                                    
⎢─────────────────────────────────────────────────────────────────────  ──────
⎢   2       2         2       2         2       2         2       2        2  
⎢sin (ψ)⋅sin (θ) + sin (ψ)⋅cos (θ) + sin (θ)⋅cos (ψ) + cos (ψ)⋅cos (θ)  sin (φ
⎢                                                                             
⎢                                                                             
⎢                            sin(ψ)⋅cos(θ)                                    
⎢─────────────────────────────────────────────────────────────────────  ──────
⎢   2       2         2       2         2       2         2       2        2  
⎢sin (ψ)⋅sin (θ) + sin (ψ)⋅cos (θ) + sin (θ)⋅cos (ψ) + cos (ψ)⋅cos (θ)  sin (φ
⎢                                                                             
⎢                               -sin(θ)             

In [12]:
# print(diff(T_rect, x))
# diff(T_rect,y)
# print(diff(T_rect, z))
print(diff(T_rect, psi))
# diff(T_rect, phi)
diff(T_rect, psi)


Matrix([[-sin(psi)*cos(theta)/(sin(psi)**2*sin(theta)**2 + sin(psi)**2*cos(theta)**2 + sin(theta)**2*cos(psi)**2 + cos(psi)**2*cos(theta)**2), (-sin(phi)*sin(psi)*sin(theta) - sin(theta)**2*cos(phi)*cos(psi) - cos(phi)*cos(psi)*cos(theta)**2)/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), (sin(phi)*sin(theta)**2*cos(psi) + sin(phi)*cos(psi)*cos(theta)**2 - sin(psi)*sin(theta)*cos(phi))/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), 0], [cos(psi)*co

⎡                                                                             
⎢                           -sin(ψ)⋅cos(θ)                                    
⎢─────────────────────────────────────────────────────────────────────  ──────
⎢   2       2         2       2         2       2         2       2        2  
⎢sin (ψ)⋅sin (θ) + sin (ψ)⋅cos (θ) + sin (θ)⋅cos (ψ) + cos (ψ)⋅cos (θ)  sin (φ
⎢                                                                             
⎢                                                                             
⎢                            cos(ψ)⋅cos(θ)                                    
⎢─────────────────────────────────────────────────────────────────────  ──────
⎢   2       2         2       2         2       2         2       2        2  
⎢sin (ψ)⋅sin (θ) + sin (ψ)⋅cos (θ) + sin (θ)⋅cos (ψ) + cos (ψ)⋅cos (θ)  sin (φ
⎢                                                                             
⎢                                  0                

In [13]:
# # print(T_rect) # -> take printed output and convert to numpy array

# # from tensorflow.math import sin, cos
# from numpy import sin, cos

# x = np.array([1,2,3])#[:,None]
# y = np.array([1,2,3])#[:,None]
# z = np.array([1,2,3])#[:,None]
# phi = np.array([1,2,3])#[:,None]
# theta = np.array([1,2,3])#[:,None]
# psi = np.array([1,2,3])#[:,None]
# test = np.array([cos(x) * cos(y) / sin(phi), cos(x) * cos(y) / sin(phi)])
# print(np.shape(test))
# print(test)

# T_rect_np = np.array([[cos(psi)*cos(theta)/(sin(psi)**2*sin(theta)**2 + sin(psi)**2*cos(theta)**2 + sin(theta)**2*cos(psi)**2 + cos(psi)**2*cos(theta)**2), (sin(phi)*sin(theta)*cos(psi) - sin(psi)*sin(theta)**2*cos(phi) - sin(psi)*cos(phi)*cos(theta)**2)/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), (sin(phi)*sin(psi)*sin(theta)**2 + sin(phi)*sin(psi)*cos(theta)**2 + sin(theta)*cos(phi)*cos(psi))/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), -x], [sin(psi)*cos(theta)/(sin(psi)**2*sin(theta)**2 + sin(psi)**2*cos(theta)**2 + sin(theta)**2*cos(psi)**2 + cos(psi)**2*cos(theta)**2), (sin(phi)*sin(psi)*sin(theta) + sin(theta)**2*cos(phi)*cos(psi) + cos(phi)*cos(psi)*cos(theta)**2)/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), (-sin(phi)*sin(theta)**2*cos(psi) - sin(phi)*cos(psi)*cos(theta)**2 + sin(psi)*sin(theta)*cos(phi))/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), -y], [-sin(theta)/(sin(theta)**2 + cos(theta)**2), sin(phi)*cos(theta)/(sin(phi)**2*sin(theta)**2 + sin(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2 + cos(phi)**2*cos(theta)**2), cos(phi)*cos(theta)/(sin(phi)**2*sin(theta)**2 + sin(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2 + cos(phi)**2*cos(theta)**2), -z], 
#                       [np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.ones(len(x))]])
# print(np.shape(T_rect_np))
# test = np.transpose(T_rect_np, (2,0,1))
# print(test)

In [14]:
# y_j = y_j[:3]
# if np.shape(y_j)[1] == 3:
#     y_j = np.append(y_j, np.ones([len(y_j),1]), axis = 1)

# # y_j_tf = tf.convert_to_tensor(y_j[:,:,None])
# # test_tf = tf.convert_to_tensor(test)
# # print(np.shape(y_j_tf))
# # print(np.shape(test_tf))

# # ans = y_j_tf * test_tf

# print(np.shape(y_j[:,:,None]))
# print(np.shape(test))
# # ans = y_j[:,:,None] * test
# ans = test @ y_j[:,:,None]


# print(np.shape(ans))
# print(ans)
# ans = np.reshape(ans, (-1, 1))
# print(ans)

In [15]:
# diff(T_rect, x)
diff(T_rect, phi)

# dT_rect_dx = diff(T_rect, x)
# pprint(dT_rect_dx)

⎡                                                                             
⎢                                                                        sin(φ
⎢0  ──────────────────────────────────────────────────────────────────────────
⎢      2       2       2         2       2       2         2       2       2  
⎢   sin (φ)⋅sin (ψ)⋅sin (θ) + sin (φ)⋅sin (ψ)⋅cos (θ) + sin (φ)⋅sin (θ)⋅cos (ψ
⎢                                                                             
⎢                                                                             
⎢                                                                       - sin(
⎢0  ──────────────────────────────────────────────────────────────────────────
⎢      2       2       2         2       2       2         2       2       2  
⎢   sin (φ)⋅sin (ψ)⋅sin (θ) + sin (φ)⋅sin (ψ)⋅cos (θ) + sin (φ)⋅sin (θ)⋅cos (ψ
⎢                                                                             
⎢                                                   

In [16]:
#substitue in values
expr = diff(T_rect, phi)
# print(expr)
# expr = T_rect
# print(m_hat)
ans = expr.subs([(x, m_hat[0]), 
                 (y, m_hat[1]),
                 (z, m_hat[2]),
                 (phi, m_hat[3]),
                 (theta, m_hat[4]),
                 (psi, m_hat[5])])
print(type(np.array(ans)))
print("\n", np.array(ans).astype(np.float64))

<class 'numpy.ndarray'>

 [[ 0.          0.58433397  0.36598239  0.        ]
 [ 0.         -0.07206591 -0.86560155  0.        ]
 [ 0.          0.80830707 -0.34174675  0.        ]
 [ 0.          0.          0.          0.        ]]


# Test rectifying motion distortion usuing SymPy Linalg technique

In [66]:
from remove_motion_basic import get_H
from remove_motion_basic import linear_correction_old as lc 

m_hat = np.array([3, 0, 0, 0, 0, 0])
skip = 1
y_j = old_cloud[::skip]
# y_j = old_cloud[::skip] + np.array([0.5, 0.5, 0.]) #test offset - hurts performance but doesn't cause anything to explode
y_i = new_cloud[::skip] #has distortion already corrected

# #remove ground plane
# y_i = y_i[y_i[:,2] > -1]
# y_j = y_j[y_j[:,2] > -1]

H = get_H(y_j = y_j, m_hat = m_hat )

#plot result
plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_j, c = "#CB2314", r=3.5))
disp.append(Points(y_i, c = "#3F5151", r=3.5))
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [59]:
print(np.shape(H))
print(np.shape(y_j)[0]*4)

(72296, 6)
72296


# Simple Test using pre-rectified point cloud
#### in this example we don't need to worry about iteratively finding correspondences
(y_i and y_j are the same length and are ordered accordingly) 

In [67]:
#assume initial correction factor of 0 for each solution component
m_hat = np.array([0., 0, 0, 0, 0, 0])
skip = 1 #50
y_i = new_cloud[::skip] #baseline
y_j = old_cloud[::skip] #distorted cloud
# y_j = old_cloud[::skip] + np.array([0.5, 0.5, 0.]) #test offset - hurts performance but doesn't cause anything to explode

# #add noise 
# noise_scale = 0.01
# y_i += np.random.randn(np.shape(y_i)[0],np.shape(y_i)[1])*noise_scale 
# y_j += np.random.randn(np.shape(y_j)[0],np.shape(y_j)[1])*noise_scale 

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_i[:,:3], c = "#2c7c94 ", alpha = 1, r=3.5))

runlen = 10
for count in range(runlen):
    
    print("~~~~ iteration ", count, "~~~~~~~~~~~")
    print("m_hat = ", m_hat)
    
    #apply last estimate of correction
    y_j_undistort = lc(y_j, m_hat)
    print(np.shape(y_j_undistort))
    
    #delta_m =  ((H^T*H)^-1)(H^T)(yi - yj*)
    H = get_H(y_j_undistort, m_hat)
    residual = (np.append(y_i, np.ones([len(y_i),1]), axis = 1) -
                np.append(y_j_undistort, np.ones([len(y_i),1]), axis = 1)).flatten()
    print("residual", np.shape(residual))
    delta_m = np.linalg.pinv(H.T @ H) @ H.T @ residual
    m_hat[:3] -= delta_m[:3]
    m_hat[3:] += delta_m[3:] #TODO: figure out why angles are backwards...    
    
    #plot updated cloud2
#     color = [0.5 + count/(runlen*2), 1 - count/runlen, count/runlen]
    disp.append(Points(y_j_undistort[:,:3], c = "#a65852 ", alpha = (count+1)/(runlen+1), r=3.5))

    
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

~~~~ iteration  0 ~~~~~~~~~~~
m_hat =  [0. 0. 0. 0. 0. 0.]
(16564, 3)
residual (66256,)
~~~~ iteration  1 ~~~~~~~~~~~
m_hat =  [-0.34599979 -0.45758962  0.00315352 -0.00617179 -0.01010173 -0.39338193]
(16564, 3)
residual (66256,)
~~~~ iteration  2 ~~~~~~~~~~~
m_hat =  [-5.21549169e-02 -1.13094770e+00  7.33291572e-04 -1.56620805e-03
 -1.44139027e-02 -6.04915934e-01]
(16564, 3)
residual (66256,)
~~~~ iteration  3 ~~~~~~~~~~~
m_hat =  [ 0.49513983 -1.56394438  0.00167887  0.00308795 -0.01433306 -0.7192103 ]
(16564, 3)
residual (66256,)
~~~~ iteration  4 ~~~~~~~~~~~
m_hat =  [ 1.07761083 -1.76336648  0.00527027  0.00596725 -0.01239772 -0.78969595]
(16564, 3)
residual (66256,)
~~~~ iteration  5 ~~~~~~~~~~~
m_hat =  [ 1.58916269 -1.80149588  0.00949982  0.00733402 -0.00997579 -0.83877971]
(16564, 3)
residual (66256,)
~~~~ iteration  6 ~~~~~~~~~~~
m_hat =  [ 1.99732988 -1.74510936  0.01309584  0.0076637  -0.00766931 -0.87570496]
(16564, 3)
residual (66256,)
~~~~ iteration  7 ~~~~~~~~~~~
m_hat

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

# Test using spherical voxels to form distributions

In [79]:
from linear_corrector import LC
#remove ground plane
pc1 = new_cloud[new_cloud[:,2] > -1]
pc2 = old_cloud[old_cloud[:,2] > -1]
m_hat0 = np.array([0.,0, 0, 0, 0, 0])
dc = LC(cloud1 = pc1, cloud2 = pc2, fid = 30, niter = 100, draw = True, m_hat0 = m_hat0)
ViewInteractiveWidget(dc.plt.window)

~~~~~~~~~~~Iteration  0 ~~~~~~~~~~
(96,)
m_hat:  [ 0.28147387  0.01675948 -0.01231744  0.00668656  0.00419819  0.00370996]
~~~~~~~~~~~Iteration  1 ~~~~~~~~~~
(104,)
m_hat:  [ 0.5825855   0.04306377 -0.01347022  0.00981566  0.00577639  0.00944217]
~~~~~~~~~~~Iteration  2 ~~~~~~~~~~
(96,)
m_hat:  [ 0.93750185  0.10821601 -0.02062513  0.01200823  0.00565066  0.01570772]
~~~~~~~~~~~Iteration  3 ~~~~~~~~~~
(92,)
m_hat:  [ 1.22032246  0.11882488 -0.02850608  0.01361919  0.00508734  0.02003206]
~~~~~~~~~~~Iteration  4 ~~~~~~~~~~
(92,)
m_hat:  [ 1.42996787  0.12596308 -0.0335662   0.01379782  0.00393005  0.01955867]
~~~~~~~~~~~Iteration  5 ~~~~~~~~~~
(92,)
m_hat:  [ 1.58328021  0.13067078 -0.03168347  0.01394258  0.0036147   0.01680094]
~~~~~~~~~~~Iteration  6 ~~~~~~~~~~
(92,)
m_hat:  [ 1.70579058  0.14116752 -0.03060816  0.0135834   0.00279026  0.01201979]
~~~~~~~~~~~Iteration  7 ~~~~~~~~~~
(92,)
m_hat:  [ 1.80568367  0.15524619 -0.03043452  0.01336227  0.0021278   0.00645975]
~~~~~~~~~~~Iter

(112,)
m_hat:  [ 5.10569618e+00 -2.75974398e+00 -5.83614838e-02 -3.15030674e-03
 -1.28425779e-02 -2.30806321e-01]
~~~~~~~~~~~Iteration  60 ~~~~~~~~~~
(112,)
m_hat:  [ 5.10996041e+00 -2.76317317e+00 -5.90034963e-02 -3.14683384e-03
 -1.28386696e-02 -2.31394413e-01]
~~~~~~~~~~~Iteration  61 ~~~~~~~~~~
(112,)
m_hat:  [ 5.11334631e+00 -2.76767932e+00 -5.93643596e-02 -3.15179981e-03
 -1.28093497e-02 -2.32173711e-01]
~~~~~~~~~~~Iteration  62 ~~~~~~~~~~
(112,)
m_hat:  [ 5.11355914e+00 -2.77359284e+00 -5.87090437e-02 -3.25764282e-03
 -1.27283658e-02 -2.33271511e-01]
~~~~~~~~~~~Iteration  63 ~~~~~~~~~~
(112,)
m_hat:  [ 5.11527159e+00 -2.77645560e+00 -5.86395038e-02 -3.21173329e-03
 -1.26698361e-02 -2.34329119e-01]
~~~~~~~~~~~Iteration  64 ~~~~~~~~~~
(112,)
m_hat:  [ 5.11361362e+00 -2.77757288e+00 -5.71348911e-02 -3.38889520e-03
 -1.26132850e-02 -2.35523033e-01]
~~~~~~~~~~~Iteration  65 ~~~~~~~~~~
(112,)
m_hat:  [ 5.11255152e+00 -2.77563410e+00 -5.71104857e-02 -3.33524489e-03
 -1.25231860e-02 -2.

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [80]:
#DEBUG: how is LC holding on to values of m_hat from the last time it was run??!

# plt2 = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
# disp=[]
# disp.append(Points(pc1,  c = 'red'))
# disp.append(Points(pc2, c = 'blue'))
# plt2.show(disp, "test viz")
# ViewInteractiveWidget(plt2.window)